#### Simulation

##### Import

In [8]:
import numpy as np
import itertools
import requests
import polyline
import json
import os

from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

from shapely.geometry import Point

import warnings 

warnings.filterwarnings('ignore')

##### 좌표 거리 생성 함수

In [2]:
def calculate_straight_distance(lat1, lon1, lat2, lon2):

    km_constant = 3959* 1.609344
    lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1 
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    km = km_constant * c
    
    return km

##### trips 데이터 생성 함수

In [5]:
# library
#############
# OSRM base #
#############
def get_res(point):

   status = 'defined'

   session = requests.Session()
   retry = Retry(connect=3, backoff_factor=0.5)
   adapter = HTTPAdapter(max_retries=retry)
   session.mount('http://', adapter)
   session.mount('https://', adapter)

   overview = '?overview=full'
   loc = f"{point[0]},{point[1]};{point[2]},{point[3]}" # lon, lat, lon, lat
   # url = "http://127.0.0.1:5000/route/v1/driving/"
   url = 'http://router.project-osrm.org/route/v1/driving/'
   r = session.get(url + loc + overview) 
   
   if r.status_code!= 200:
      
      status = 'undefined'
      
      # distance    
      distance = calculate_straight_distance(point[1], point[0], point[3], point[2]) * 1000
      
      # route
      route = [[point[0], point[1]], [point[2], point[3]]]

      # duration & timestamp
      speed_km = 10#km
      speed = (speed_km * 1000/60)      
      duration = distance/speed
      
      timestamp = [0, duration]

      result = {'route': route, 'timestamp': timestamp, 'duration': duration, 'distance' : distance}
   
      return result, status
   
   res = r.json()   
   return res, status

##############################
# Extract duration, distance #
##############################
def extract_duration_distance(res):
       
   duration = res['routes'][0]['duration']/60
   distance = res['routes'][0]['distance']
   
   return duration, distance

#################
# Extract route #
#################
def extract_route(res):
    
    route = polyline.decode(res['routes'][0]['geometry'])
    route = list(map(lambda data: [data[1],data[0]] ,route))
    
    return route

#####################
# Extract timestamp #
#####################
def extract_timestamp(route, duration):
    
    rt = np.array(route)
    rt = np.hstack([rt[:-1,:], rt[1:,:]])

    per = calculate_straight_distance(rt[:,1], rt[:,0], rt[:,3], rt[:,2])
    per = per / np.sum(per)

    timestamp = per * duration
    timestamp = np.hstack([np.array([0]),timestamp])
    timestamp = list(itertools.accumulate(timestamp)) 
    
    return timestamp
 
########
# MAIN #
########

# - input : O_point, D_point (shapely.geometry.Point type)
# - output : trip, timestamp, duration, distance
def osrm_routing_machine(O, D):
       
   osrm_base, status = get_res([O.x, O.y, D.x, D.y])
   
   if status == 'defined':
      duration, distance = extract_duration_distance(osrm_base)
      route = extract_route(osrm_base)
      timestamp = extract_timestamp(route, duration)

      result = {'route': route, 'timestamp': timestamp, 'duration': duration, 'distance' : distance}
      
      return result
   else: 
      return osrm_base

def osrm_routing_machine_multiprocess(OD):
   O, D = OD
   result = osrm_routing_machine(O, D)
   return result

def osrm_routing_machine_multiprocess_all(OD_data):
    results = list(map(osrm_routing_machine_multiprocess, OD_data))
    return results

In [4]:
가천대역 = Point(127.126858, 37.449621)
가천대_반도체대학 = Point(127.127384 , 37.450910)
가천대_일반대학원 = Point(127.130112 , 37.452589)
가천대_교육대학원 = Point(127.131698 , 37.452066)
가천대_학생회관 = Point(127.134042 , 37.453336)
가천대_ai_공학관 = Point(127.133374 , 37.455009)

In [6]:
result = osrm_routing_machine(가천대역,가천대_ai_공학관)

In [13]:
result

{'route': [[127.12681, 37.44962],
  [127.12679, 37.45052],
  [127.12692, 37.45052],
  [127.12742, 37.45052],
  [127.12749, 37.45052],
  [127.1275, 37.45052],
  [127.12754, 37.45051],
  [127.12782, 37.45042],
  [127.12786, 37.45042],
  [127.12798, 37.45041],
  [127.12914, 37.4501],
  [127.12916, 37.45009],
  [127.12952, 37.44999],
  [127.12956, 37.44998],
  [127.12964, 37.44996],
  [127.12966, 37.44997],
  [127.12974, 37.44997],
  [127.12982, 37.45],
  [127.12988, 37.45005],
  [127.12993, 37.45013],
  [127.13007, 37.45045],
  [127.13012, 37.45056],
  [127.13016, 37.45066],
  [127.1302, 37.45085],
  [127.13027, 37.45095],
  [127.13037, 37.45101],
  [127.13052, 37.45108],
  [127.1307, 37.45115],
  [127.1308, 37.45122],
  [127.13087, 37.4513],
  [127.13092, 37.45139],
  [127.13099, 37.45153],
  [127.13114, 37.45179],
  [127.13125, 37.45197],
  [127.13131, 37.45206],
  [127.13137, 37.45216],
  [127.13144, 37.45222],
  [127.13152, 37.45226],
  [127.13161, 37.45229],
  [127.13173, 37.45232],


In [12]:
path = '../simulation/src/data/'

with open(os.path.join(path + 'trips.json'), 'w', encoding='utf-8') as file:
    json.dump(result, file)